## GOLD LAYER — Business-Ready Analytics Tables
Purpose of Gold

Store final, business-facing tables used directly by Power BI and SQL analysis.
This is where business logic lives.

##### Gold KPI Table (gold.daily_kpis) — Explanation

What this code does

Aggregates order-level data by day
Calculates executive KPIs:
Total Orders
Ordering Users
Net Revenue (Net GMV)
Average Order Value (AOV)

Why this table exists

Executives and analysts need daily performance metrics
Removes the need for complex calculations in Power BI
Ensures KPI definitions are consistent

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# ----------------------------
# 0) Create Gold database
# ----------------------------
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

# ----------------------------
# 1) Read Silver tables
# ----------------------------
customers   = spark.table("silver.customers")
sessions    = spark.table("silver.sessions")
events      = spark.table("silver.events")
products    = spark.table("silver.products")
orders      = spark.table("silver.orders")
order_items = spark.table("silver.order_items")

from pyspark.sql import functions as F

spark.sql("CREATE DATABASE IF NOT EXISTS gold")

orders = spark.table("silver.orders")

daily_kpis = (
    orders
    .where(F.col("order_date").isNotNull())
    .groupBy("order_date")
    .agg(
        F.countDistinct("order_id").alias("orders"),
        F.countDistinct("user_id").alias("ordering_users"),
        F.sum("net_gmv").alias("net_gmv"),
    )
    .withColumn(
        "aov",
        F.when(F.col("orders") == 0, F.lit(None)).otherwise(F.col("net_gmv") / F.col("orders"))
    )
)

(daily_kpis.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("gold.daily_kpis")
)


StatementMeta(, 9d1c53e9-0d9d-4b13-9a67-baf700570694, 4, Finished, Available, Finished)

In [3]:
spark.table("gold.daily_kpis").orderBy(F.desc("order_date")).show(10, truncate=False)


StatementMeta(, 9d1c53e9-0d9d-4b13-9a67-baf700570694, 5, Finished, Available, Finished)

+----------+------+--------------+-------+----------------+
|order_date|orders|ordering_users|net_gmv|aov             |
+----------+------+--------------+-------+----------------+
|2025-10-31|16    |16            |1765.02|110.313750000000|
|2025-10-30|12    |12            |2928.40|244.033333333333|
|2025-10-29|14    |14            |3174.19|226.727857142857|
|2025-10-28|17    |17            |2462.59|144.858235294118|
|2025-10-27|12    |12            |1582.32|131.860000000000|
|2025-10-26|16    |16            |2309.82|144.363750000000|
|2025-10-25|16    |16            |2422.35|151.396875000000|
|2025-10-24|18    |18            |1764.84|98.046666666667 |
|2025-10-23|23    |23            |3088.11|134.265652173913|
|2025-10-22|17    |17            |1906.14|112.125882352941|
+----------+------+--------------+-------+----------------+
only showing top 10 rows



In [4]:
spark.table("silver.events").columns


StatementMeta(, 9d1c53e9-0d9d-4b13-9a67-baf700570694, 6, Finished, Available, Finished)

['event_id',
 'session_id',
 'event_time',
 'event_type',
 'product_id',
 'qty',
 'cart_size',
 'payment',
 'discount_pct',
 'amount_usd',
 'event_date']

##### Gold: Session-Level Funnel Flags (gold.session_funnel_flags)

What this code does

Creates a session-level funnel table,
Flags whether each session reached key funnel stages:
did_view,
did_add_to_cart,
did_checkout,
did_purchase.

Each flag is binary (1 = reached the stage, 0 = did not)

Why this table exists

Allows funnel analysis at the session level.
Makes it easy to calculate:
Conversion rates,
Drop-offs between stages,
Simplifies Power BI modeling by avoiding complex event-level logic.

How it is used

Sessions are counted once per stage (no double counting),
Downstream Gold tables aggregate these flags by day.

Power BI uses this table to:

Visualize funnels,
Compare stage-to-stage leakage.

In [1]:
from pyspark.sql import functions as F

spark.sql("CREATE DATABASE IF NOT EXISTS gold")

sessions = spark.table("silver.sessions")
events = spark.table("silver.events")

# ---- Event type mapping (adjust only if your dataset uses different labels)
evt = F.col("event_type")

is_view = evt.isin("view", "page_view", "product_view")
is_cart = evt.isin("add_to_cart", "cart", "addtocart")
is_checkout = evt.isin("checkout", "checkout_start", "begin_checkout")
is_purchase_by_type = evt.isin("purchase", "payment_success", "order_placed", "transaction")

# Fallback purchase signal: payment present (string not null/empty) or payment == '1'/'true'
is_purchase_by_payment = (
    F.col("payment").isNotNull() &
    (F.trim(F.col("payment").cast("string")) != F.lit(""))
)

events_flags = (
    events
    .groupBy("session_id")
    .agg(
        F.max(F.when(is_view, 1).otherwise(0)).alias("did_view"),
        F.max(F.when(is_cart, 1).otherwise(0)).alias("did_add_to_cart"),
        F.max(F.when(is_checkout, 1).otherwise(0)).alias("did_checkout"),
        F.max(F.when(is_purchase_by_type | is_purchase_by_payment, 1).otherwise(0)).alias("did_purchase")
    )
)

session_funnel_flags = (
    sessions
    .select(
        "session_id",
        "user_id",
        "session_start",
        "session_date",
        *([c for c in ["device_type"] if c in sessions.columns])
    )
    .join(events_flags, on="session_id", how="left")
    .fillna(0, subset=["did_view","did_add_to_cart","did_checkout","did_purchase"])
    .withColumn("visited", F.lit(1))
)

(session_funnel_flags.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("gold.session_funnel_flags")
)


StatementMeta(, 378bda87-6c01-452d-8bb5-ce2091677bd4, 3, Finished, Available, Finished)

In [2]:
spark.table("gold.session_funnel_flags").groupBy("visited","did_view","did_add_to_cart","did_checkout","did_purchase").count().show(20, truncate=False)


StatementMeta(, 378bda87-6c01-452d-8bb5-ce2091677bd4, 4, Finished, Available, Finished)

+-------+--------+---------------+------------+------------+-----+
|visited|did_view|did_add_to_cart|did_checkout|did_purchase|count|
+-------+--------+---------------+------------+------------+-----+
|1      |1       |1              |0           |0           |36609|
|1      |1       |1              |1           |1           |33580|
|1      |1       |1              |1           |0           |11329|
|1      |1       |0              |0           |0           |38482|
+-------+--------+---------------+------------+------------+-----+



##### Gold Funnel Table (gold.funnel_summary_daily) — Explanation

What this code does

Aggregates session-level behavior into funnel stages
Tracks how many sessions reach each stage:
visited 
view 
add_to_cart 
checkout 
purchase 

Calculates daily funnel volumes

What this enables

Funnel visualization in Power BI
Conversion rate calculations using DAX
Leakage analysis between stages

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

base = spark.table("gold.session_funnel_flags")

group_cols = ["session_date"]
if "device_type" in base.columns:
    group_cols.append("device_type")

stage_counts = (
    base.groupBy(group_cols)
    .agg(
        F.countDistinct("session_id").alias("visited_sessions"),
        F.countDistinct(F.when(F.col("did_view") == 1, F.col("session_id"))).alias("view_sessions"),
        F.countDistinct(F.when(F.col("did_add_to_cart") == 1, F.col("session_id"))).alias("cart_sessions"),
        F.countDistinct(F.when(F.col("did_checkout") == 1, F.col("session_id"))).alias("checkout_sessions"),
        F.countDistinct(F.when(F.col("did_purchase") == 1, F.col("session_id"))).alias("purchase_sessions")
    )
)

funnel_long = (
    stage_counts
    .select(
        *group_cols,
        F.expr(
            "stack(5, "
            "'visited', visited_sessions, "
            "'view', view_sessions, "
            "'add_to_cart', cart_sessions, "
            "'checkout', checkout_sessions, "
            "'purchase', purchase_sessions"
            ") as (stage, sessions)"
        )
    )
)

stage_rank = (
    F.when(F.col("stage") == "visited", 1)
     .when(F.col("stage") == "view", 2)
     .when(F.col("stage") == "add_to_cart", 3)
     .when(F.col("stage") == "checkout", 4)
     .when(F.col("stage") == "purchase", 5)
     .otherwise(99)
)

w = Window.partitionBy(*group_cols).orderBy(stage_rank)

funnel_summary_daily = (
    funnel_long
    .withColumn("stage_rank", stage_rank)
    .withColumn("prev_sessions", F.lag("sessions").over(w))
    .withColumn("start_sessions", F.first("sessions").over(w))
    .withColumn(
        "pct_from_prev",
        F.when((F.col("prev_sessions").isNull()) | (F.col("prev_sessions") == 0), F.lit(None))
         .otherwise(F.round(100 * F.col("sessions") / F.col("prev_sessions"), 2))
    )
    .withColumn(
        "pct_from_start",
        F.when(F.col("start_sessions") == 0, F.lit(None))
         .otherwise(F.round(100 * F.col("sessions") / F.col("start_sessions"), 2))
    )
    .drop("stage_rank")
)

(funnel_summary_daily.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("gold.funnel_summary_daily")
)


StatementMeta(, 378bda87-6c01-452d-8bb5-ce2091677bd4, 6, Finished, Available, Finished)

In [5]:
spark.table("gold.funnel_summary_daily").orderBy(F.desc("session_date"), "stage").show(30, truncate=False)


StatementMeta(, 378bda87-6c01-452d-8bb5-ce2091677bd4, 7, Finished, Available, Finished)

+------------+-----------+-----------+--------+-------------+--------------+-------------+--------------+
|session_date|device_type|stage      |sessions|prev_sessions|start_sessions|pct_from_prev|pct_from_start|
+------------+-----------+-----------+--------+-------------+--------------+-------------+--------------+
|2025-10-31  |desktop    |add_to_cart|12      |16           |16            |75.0         |75.0          |
|2025-10-31  |mobile     |add_to_cart|22      |32           |32            |68.75        |68.75         |
|2025-10-31  |tablet     |add_to_cart|2       |6            |6             |33.33        |33.33         |
|2025-10-31  |desktop    |checkout   |5       |12           |16            |41.67        |31.25         |
|2025-10-31  |mobile     |checkout   |12      |22           |32            |54.55        |37.5          |
|2025-10-31  |tablet     |checkout   |0       |2            |6             |0.0          |0.0           |
|2025-10-31  |mobile     |purchase   |11      

##### Gold Product Sales Table (gold.product_sales_daily) — Explanation
What this code does

Aggregates purchase events by day

Captures:
Revenue,
Quantity sold,
Attempts to associate revenue with products.

Important limitation (documented intentionally)

Product identifiers are missing or null for most rows,
Product-level analysis is therefore limited,
This is explicitly documented as a data quality finding.

Why we still keep this table

Enables revenue attribution checks,
Helps identify instrumentation gaps,
Supports future enhancements once attribution is fixed.

In [6]:
from pyspark.sql import functions as F, types as T

events = spark.table("silver.events")
products = spark.table("silver.products")

purchase_events = events.where(
    (F.col("event_type").isin("purchase", "payment_success", "order_placed", "transaction")) |
    (F.col("payment").isNotNull() & (F.trim(F.col("payment").cast("string")) != ""))
)

product_sales_daily = (
    purchase_events
    .withColumn("qty", F.coalesce(F.col("qty").cast("int"), F.lit(1)))
    .withColumn("amount_usd_num", F.regexp_replace(F.col("amount_usd"), r"[^0-9\.\-]", "").cast(T.DecimalType(18,2)))
    .groupBy("event_date", "product_id")
    .agg(
        F.sum("qty").alias("qty"),
        F.sum("amount_usd_num").alias("gross_sales_usd")
    )
    .join(products.select("product_id", *[c for c in ["title","category","brand_name"] if c in products.columns]),
          on="product_id",
          how="left")
)

(product_sales_daily.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("gold.product_sales_daily")
)


StatementMeta(, 378bda87-6c01-452d-8bb5-ce2091677bd4, 8, Finished, Available, Finished)

In [7]:
spark.table("gold.product_sales_daily").orderBy(F.desc("event_date")).show(20, truncate=False)


StatementMeta(, 378bda87-6c01-452d-8bb5-ce2091677bd4, 9, Finished, Available, Finished)

+----------+----------+---+---------------+-----+--------+
|product_id|event_date|qty|gross_sales_usd|title|category|
+----------+----------+---+---------------+-----+--------+
|NULL      |2025-10-31|16 |1765.02        |NULL |NULL    |
|NULL      |2025-10-30|12 |2928.40        |NULL |NULL    |
|NULL      |2025-10-29|14 |3174.19        |NULL |NULL    |
|NULL      |2025-10-28|17 |2462.59        |NULL |NULL    |
|NULL      |2025-10-27|12 |1582.32        |NULL |NULL    |
|NULL      |2025-10-26|16 |2309.82        |NULL |NULL    |
|NULL      |2025-10-25|16 |2422.35        |NULL |NULL    |
|NULL      |2025-10-24|18 |1764.84        |NULL |NULL    |
|NULL      |2025-10-23|23 |3088.11        |NULL |NULL    |
|NULL      |2025-10-22|17 |1906.14        |NULL |NULL    |
|NULL      |2025-10-21|19 |3083.50        |NULL |NULL    |
|NULL      |2025-10-20|12 |1844.02        |NULL |NULL    |
|NULL      |2025-10-19|15 |1537.93        |NULL |NULL    |
|NULL      |2025-10-18|14 |2328.02        |NULL |NULL   